Visualizer for simulated dynamic adaptive networks

Access: datafiles with adj matrices+culture vecs

In [22]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import extractDat as exdat
from mpl_toolkits.mplot3d import Axes3D

In [9]:
df = exdat.read_pickle('df_Sayama')
df.head()

,degrees,clusterCoeff,reciprocity,giantComponent,diam,SPL,CD,std_d,std_rs,std_rw,tags
0,"[23, 22, 21, 20, 20, 19, 19, 18, 18, 18, 17, 1...",0.261858,0.246575,1.0,4,2.44163,2.853668,0.0,0.0,0.0,0
1,"[22, 22, 20, 19, 19, 18, 18, 18, 18, 17, 17, 1...",0.235789,0.220386,1.0,3,2.39592,2.248453,0.0,0.0,0.0,1
2,"[22, 22, 22, 21, 21, 21, 20, 20, 19, 19, 19, 1...",0.278229,0.256000,1.0,3,2.4351,2.910656,0.0,0.0,0.0,2
3,"[25, 22, 21, 21, 21, 21, 21, 20, 20, 20, 19, 1...",0.277090,0.298734,1.0,3,2.36204,2.406050,0.0,0.0,0.0,3
4,"[22, 21, 21, 19, 19, 18, 18, 17, 17, 17, 17, 1...",0.255338,0.237569,1.0,3,2.46939,2.888474,0.0,0.0,0.0,4


In [7]:
con = df['giantComponent'] < 1.0
df.loc[con].shape

(599, 11)

In [11]:
df_full = df
df = df.loc[df['giantComponent'] == 1.0]
print(df.shape, df_full.shape)
df.columns



(21001, 11) (21001, 11)


Index(['degrees', 'clusterCoeff', 'reciprocity', 'giantComponent', 'diam',
       'SPL', 'CD', 'std_d', 'std_rs', 'std_rw', 'tags'],
      dtype='object')

In [42]:
mask = df['SPL'].apply(lambda x: isinstance(x, tuple))
df = df[~mask]
print(df.shape, df_full.shape)mask = df['SPL'].apply(lambda x: isinstance(x, tuple))
df = df[~mask]
print(df.shape, df_full.shape)

(21001, 11) (21001, 11)


In [43]:
stdd, stdrs, stdrw = df['std_d'].values, df['std_rs'].values, df['std_rw'].values
spl = df['SPL'].values
cd = df['CD'].values


In [44]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(stdd, cd, spl)
plt.ylabel("<CD>")
#plt.zlabel("<SPL>")
plt.xlabel("s.d. of d")

Text(0.5, 0, 's.d. of d')

ValueError: Invalid RGBA argument: masked_array(data=[0.12156862745098039, 0.4666666666666667,
                   0.7058823529411765, 0.3713527886674658],
             mask=False,
       fill_value='?',
            dtype=object)

<Figure size 432x288 with 1 Axes>

In [41]:
tuples = type(df['SPL']) == tuple
len(tuples)

TypeError: object of type 'bool' has no len()